# Tutoriel sur la préparation des données

## 1. Lecture des données

Pour lire les données, on va utiliser la bibliothèque **pandas**.
Elle support [plusieurs types de fichiers](https://pandas.pydata.org/pandas-docs/stable/io.html): csv (read\_csv), JSON (read\_json), HTML (read\_html), MS Excel (read\_excel), SQL (read\_sql), etc.


In [704]:
import pandas

Ici, on va utiliser l'ensemble des données [Census Income Data Set (Adult)](https://archive.ics.uci.edu/ml/datasets/Census+Income).
Les données se composent de 14 caractéristiques et de 48842 échantillons.
Dans le but de l'exercice, on a réduit le nombre des caractéristiques à 7 et quelques échantillons dispersés sur plusieurs formats de fichiers.

### 1.1. Lecture du fichier adult1.csv

Le premier fichier (data/adult1.csv) est un fichier CSV avec des colonnes séparées par des virgules (50 échantillons).
Le fichier contient les colonnes suivantes (avec l'entête: titres des colonnes) dans l'ordre:
1. age: entier.
1. workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.  
1. education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.  
1. Marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
1. occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.  
1. sex: Female, Male.
1. Hours-per-week: entier.
1. class: <=50K, >50K

On sait que le fichier est bien formé; donc, on ne va pas vérifier le format.
On va ignorer les espaces qui suivent les séparateurs en utilisant l'option *skipinitialspace*.

In [705]:
adult1 = pandas.read_csv("data/adult1.csv", skipinitialspace=True)
adult1.head(3)

,age,workclass,education,Marital-status,occupation,sex,Hours-per-week,class
0,39.0,State-gov,Bachelors,Never-married,Adm-clerical,Male,40,<=50K
1,50.0,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Male,13,<=50K
2,38.0,Private,HS-grad,Divorced,Handlers-cleaners,Male,40,<=50K


### 1.2. Lecture du fichier adult2.csv

Le deuxième fichier est un fichier CSV lui aussi, mais les colonnes sont séparées par des points-virgules.
Le fichier est mal-formé; il existe des lignes avec séparation par virgules.
Aussi, il n'y a pas d'entête pour désigner les noms des colonnes (caractéristiques).
Voici le sens des colonnes dans l'ordre:
1. class: Y, N (il gagne plus de 50K)
1. age: entier
1. sex: F, M
1. workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
1. education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
1. hours-per-week: entier.
1. marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

Dans ce cas, on va créer une liste des noms des colonnes (*noms*) et l'assigner comme entête en utilisant l'option *names*.
Aussi, il faut spécifier quil n'y a pas d'entête (la première ligne contient des données et pas les noms des colonnes) en utilisant l'option *header=None*.
Le séparateur du fichier CSV peut être spécifié en utilisant l'option *sep*.

In [706]:
noms = ["class", "age", "sex", "workclass", "education", "hours-per-week", "marital-status"]
adult2 = pandas.read_csv("data/adult2.csv", skipinitialspace=True, sep=";", header=None, names=noms)
adult2.head(3)

,class,age,sex,workclass,education,hours-per-week,marital-status
0,N,25,F,Private,Some-college,40,Married-civ-spouse
1,N,18,F,Private,HS-grad,30,Never-married
2,Y,47,F,"Private, Prof-school",60,Married-civ-spouse,NaN


Pour plus d'options veuillez consulter la documentation de [read_csv](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html).
Le problème qui se pose est qu'on va avoir des lignes avec des "NaN" (valeurs non définies).
On va régler ça dans l'étape de nétoyage des données.

### Lecture du fichier adult3.db (sqlite)

Le troisième fichier est de format Sqlite, dont le schéma est le suivant:

```sql
CREATE TABLE `income` (
	`num`	INTEGER, --identifiant
	`age`	INTEGER,
	`workclass`	TEXT,
	`education`	TEXT,
	`marital-status`	TEXT,
	`sex`	TEXT,
	`hours-per-day`	REAL,
	`class`	TEXT
);
```

Les valeurs possibles des champs (les valeurs non définies sont marquées par "?"):
1. num: entier pour identifier l'individu
1. age: entier
1. workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
1. education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
1. marital-status: married, divorced, widowed, single.
1. sex: F, M
1. hours-per-day: réel; la moyenne des heurs pour chaque jour (supposant, on travaille 5 jours/semaine)
1. class: Y, N

Pour lire les données d'une base de données, on va utiliser la méthode **read_sql_query** de pandas.
L'SGBD peut être interrogé en utilisant le module **sqlite3**.
Les valeurs "?" veulent dire "pas définies".
Pour être cohérent avec les données précédentes, on doit remplacer les "?" par la valeur "NaN" de **numpy**.


In [707]:
import sqlite3
import numpy
#établir la connexion avec la base de données
con = sqlite3.connect("data/adult3.db")
#récupérer le résultat d'une réquête SQL sur cette connexion
adult3 = pandas.read_sql_query("SELECT * FROM income", con)

#remplacer les valeurs "?" par NaN de numpy
adult3 = adult3.replace('?', numpy.nan)

adult3.head(3)

,num,age,workclass,education,marital-status,sex,hours-per-day,class
0,1,76,Private,Masters,married,M,8.0,Y
1,2,44,Private,Bachelors,married,M,12.0,Y
2,3,47,Self-emp-not-inc,Masters,single,F,10.0,N


### Lecture du fichier adult4.xml

Le 4ième fichier est de format XML dont la [DTD](https://fr.wikipedia.org/wiki/Document_type_definition) est la suivante:

```xml
<?xml version="1.0" encoding="UTF-8"?>
<!ELEMENT income (candidat)*>
<!ATTLIST candidat id ID #REQUIRED>
<!ELEMENT candidat (age, workclass, education, marital-status, sex, hours-per-week, class)>
<!ELEMENT age #PCDATA>
<!ELEMENT workclass #PCDATA>
<!ELEMENT education #PCDATA>
<!ELEMENT marital-status #PCDATA>
<!ELEMENT sex #PCDATA>
<!ELEMENT hours-per-week #PCDATA>
<!ELEMENT class #PCDATA>
```

- Les valeurs possibles des champs nominaux sont comme celles de la base de données sqlite (fichier adult3.db).
- Les valeurs non définies sont représentées par l'absence de leurs balises respectives dans le fichier XML.

Ce dernier point pose un problème puisque le DTD spécifie que les champs doivent tous être présents. Donc, on doi changer le DTD afin d'accepter des champs manquants. 

```xml
<!ELEMENT candidat (age?, workclass?, education?, marital-status?, sex?, hours-per-week?, class)>
```

Pour valider le fichier XML, on va utiliser la bibliothèque **lxml**.


In [708]:
from lxml import etree
#créer le parser et spécifier qu'il doit valider le DTD
parser = etree.XMLParser(dtd_validation=True)
#analyser le fichier XML en utilisant ce parser
arbre = etree.parse("data/adult4.xml", parser)

Pour traiter un nœud XML, on va définir une fonction qui retourne son texte s'il existe, sinon la valeur "NaN" de **numpy**.

In [709]:
def valeur_noeud(noeud):
    return noeud.text if noeud is not None else numpy.nan

On crée un objet de type **pandas.DataFrame** avec les titres des colonnes.
Ensuite, on parcourt les éléments un par un en ajoutant les valeurs dans l'objet qu'on a créé.

In [710]:
noms2 = ["id", "age", "workclass", "education", "marital-status", "sex", "hours-per-week", "class"]
adult4 = pandas.DataFrame(columns=noms2)

for candidat in arbre.getroot():
    idi = candidat.get("id")
    age = valeur_noeud(candidat.find("age"))
    workclass = valeur_noeud(candidat.find("workclass"))
    education = valeur_noeud(candidat.find("education"))
    marital = valeur_noeud(candidat.find("marital-status"))
    sex = valeur_noeud(candidat.find("sex"))
    hours = valeur_noeud(candidat.find("hours-per-week"))
    klass = valeur_noeud(candidat.find("class"))

    adult4 = adult4.append(
        pandas.Series([idi, age, workclass, education, marital, sex, hours, klass],
        index=noms2), ignore_index=True)
adult4.head(3)

,id,age,workclass,education,marital-status,sex,hours-per-week,class
0,52,47,Local-gov,Some-college,divorced,F,38,N
1,53,34,Private,HS-grad,single,F,40,N
2,54,33,Private,Bachelors,single,F,40,N


## 2. Intégration des données

Lors de la lecture des 4 fichiers, on remarque que leurs schémas sont différents.
Donc, avant de fusionner les 4 schémas, il faut régler les problèmes qui gênent à cette opération.
Voici la liste des problèmes rencontrés:

1. Ordre et noms différents des caractéristiques. Par exemple, le champs "class" est en dernier dans des schémas, et en premier dans d'autres.
  - Renommer les caractéristiques
  - Réorganiser l'ordre des caractéristiques dans les tableaux.

1. Problème d'échelle: dans la caractéristique "adult3.hours-per-day" qui est représetée par "hours-per-week" dans les autres schémas. On multiplie les valeurs par 5 (nous avons supposé 5 jours/semaines) et on renomme la colonne "hours-per-week".

1. Échantillons (enregistrement) redondants: les schémas de "adult3.db" et "adult4.xml" contiennent une caractéristique: "num" et "id" respectivement (qu'on a unifié dans l'étape précédente).
Ici, on ne veut pas qu'une personne se répète plus d'une fois.
  - Supprimer une des deux échantillons redondants
  - On remarque qu'il y a des échantillons dupliqués où un est plus complet (ne contient pas de valeurs manquantes) que l'autre. Donc, on garde le plus complet.
1. Caractéristiques inutiles (de plus): adult1.csv contient la caractéristique "occupation" qui ne figure pas chez les autres fichiers.
  - Supprimer la colonne
1. Conflits de valeurs, les caractéristiques suivantes ont des différentes valeurs possibles entre les schémas (solution: unifier les valeurs).
  - marital-status: on garde les valeurs les plus restreintes (married, divorced, widowed, single). Les autres valeurs seront transformées à une de ces 4.
  - sex: on garde les valeurs avec moins de taille (F, M)
  - class: on garde les valeurs avec moins de taille (Y, N)


### 2.1. Ordre et noms différents des caractéristiques

On commence par renommer les caractéristiques identiques.
- adult3.num sera adult3.id, pour être homogène avec adult4.id
- adult3.hours-per-day sera adult3.hours-per-week pour être homogène avec les autres schémas. On va transformer les valeurs après.
- adult1.Hours-per-week sera adult1.hours-per-week (un problème de majuscule)
- adult1.Marital-status sera adult1.marital-status (un problème de majuscule)

On va utiliser la méthode [pandas.DataFrame.rename](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html) où l'objet "adult3" est de type **pandas.DataFrame**.


In [711]:
# les colonnes adult3 avant renomage
list(adult3.columns)

['num',
 'age',
 'workclass',
 'education',
 'marital-status',
 'sex',
 'hours-per-day',
 'class']

In [712]:
adult3.rename(columns={"num": "id", "hours-per-day": "hours-per-week"}, inplace=True)
adult1.rename(columns={"Hours-per-week": "hours-per-week", "Marital-status": "marital-status"}, inplace=True)

# les colonnes adult3 après renomage
list(adult3.columns)

['id',
 'age',
 'workclass',
 'education',
 'marital-status',
 'sex',
 'hours-per-week',
 'class']

Ensuite, on va ordonner les caractéristiques selon cet ordre: "age", "workclass", "education", "marital-status", "sex", "hours-per-week", "class". Les caractéristiques en plus vont être mises en derniers. On va utiliser la méthode [pandas.DataFrame.reindex](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reindex.html).

In [713]:
ordre = ["age", "workclass", "education", "marital-status", "sex", "hours-per-week", "class"]
adult1 = adult1.reindex(ordre + ["occupation"], axis=1)
#print adult1.head()
adult2 = adult2.reindex(ordre, axis=1)
adult3 = adult3.reindex(ordre + ["id"], axis=1)
adult4 = adult4.reindex(ordre + ["id"], axis=1)

#les colonnes adult3 après ordonnancement
list(adult3.columns)

['age',
 'workclass',
 'education',
 'marital-status',
 'sex',
 'hours-per-week',
 'class',
 'id']

### 2.2. Problème d'échelle

On a modifié "adult3.hours-per-day" par "hours-per-week" qui change le sens mais pas les valeurs.
On va régler ça ici.

On règle ce problème avant de régler la redondance, puisque dans l'étape suivante on va fusionner les deux tables "adult3" et "adult4".

In [714]:
# adult3 avant la transformation de l'échelle
adult3.head(2)

,age,workclass,education,marital-status,sex,hours-per-week,class,id
0,76,Private,Masters,married,M,8.0,Y,1
1,44,Private,Bachelors,married,M,12.0,Y,2


In [715]:
#transformer les heurs/jour à heurs/semaine
adult3["hours-per-week"] *= 5

# adult3 après la transformation de l'échelle
adult3.head(2)

,age,workclass,education,marital-status,sex,hours-per-week,class,id
0,76,Private,Masters,married,M,40.0,Y,1
1,44,Private,Bachelors,married,M,60.0,Y,2


### 2.3. Echantillons (enregistrement) redondants

Les tables "adult3" et "adult4" contiennent des enregistrements avec le même "id". Une solution est de fusionner les deux tables dans une seule (disant "adult34"), ensuite utiliser la méthode [pandas.DataFrame.drop_duplicates](http://pandas.pydata.org/pandas-docs/version/0.17/generated/pandas.DataFrame.drop_duplicates.html). On peut choisir quelle occurence on veut garder.

On fusionne les deux dataframes en une seule. 
Ensuite, on vérifie s'il y a des échantillons avec le même "id". Pour ce faire, on ordonne les échantillons selon le "id". Mais avant, il faut le transformer en entier


In [716]:
adult34 = pandas.concat([adult3, adult4], ignore_index=True)
# définir le type de "id" comme étant entier, et remplacer la colonne
adult34["id"] = pandas.to_numeric(adult34["id"], downcast="integer")
# ordonner les enregistrements par "id"
# il faut ordonner pour que les fonctions duplicated() et groupby() marchent
adult34 = adult34.sort_values(by="id")

# adult34 : vérification des échantillons redondants
red = adult34[adult34.duplicated("id", keep=False)]
red

,age,workclass,education,marital-status,sex,hours-per-week,class,id
44,70,Private,Some-college,single,M,40,N,45
94,70,Private,Some-college,single,M,8,N,45
45,31,Private,HS-grad,single,F,30,N,46
95,31,Private,HS-grad,single,NaN,6,N,46
46,22,Private,Some-college,married,M,24,N,47
96,22,Private,Some-college,married,M,4.8,N,47
47,36,Private,HS-grad,widowed,F,24,N,48
97,NaN,Private,HS-grad,widowed,F,4.8,N,48
48,64,Private,11th,married,M,40,N,49
98,64,Private,11th,married,M,8,N,49


Avant de supprimer un échantillon, il faut remplir les valeurs manquantes à partir des autres échaantillons identiques (le même "id") avant de supprimer un des deux. L'idée est la suivante:
- On fusionne les deux tables
- On regroupe les enregistrements en se basant sur la caractéristique (champs, attribut, colonne) "id".
- On fait un remplissage en arrière par groupe: la valeur "NaN" sera remplacée par une valeur

In [717]:
# regrouper les par "id", et pour chaque groupe remplacer les
# valeurs absentes par une valeur précédente dans le même groupe
adult34 = adult34.groupby("id").ffill()
# supprimer les enregistrements dupliqués
# on garde les derniers, puisqu'ils sont été réglés
adult34.drop_duplicates("id", keep="last", inplace=True)

# adult34 : vérification des échantillons redondants
red = adult34[adult34.duplicated("id", keep=False)]
red

,id,age,workclass,education,marital-status,sex,hours-per-week,class


### 2.4 Caractéristiques inutiles

Ici on va supprimer les colonnes inutiles:
- adult1.occupation
- adult34.id


In [718]:
# les colonnes de adult1 avant
list(adult1.columns)

['age',
 'workclass',
 'education',
 'marital-status',
 'sex',
 'hours-per-week',
 'class',
 'occupation']

In [719]:
adult1.drop(["occupation"], axis=1, inplace=True)
adult34.drop(["id"], axis=1, inplace=True)
# les colonnes de adult1 après
list(adult1.columns)

['age',
 'workclass',
 'education',
 'marital-status',
 'sex',
 'hours-per-week',
 'class']

### 2.5. Conflits de valeurs

Remplacer les valeurs de "marital-status" dans les tables "adult1" et "adult2" comme suit:
- "Never-married" par "single"
- "Married-civ-spouse", "Married-spouse-absent" et "Married-AF-spouse"  par "married"
- "Divorced" et "Separated" par "divorced"
- "Widowed" par "widowed"

On peut vérifier les valeurs possibles pour une colonne en utilisant la fonction [pandas.Series.unique](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html):

In [720]:
# les différentes valeurs du colonne adult1.marital-status
adult1["marital-status"].unique()

array(['Never-married', 'Married-civ-spouse', 'Divorced', nan,
       'Married-spouse-absent', 'Separated', 'Married-AF-spouse'],
      dtype=object)

In [721]:
dic = {
    "Never-married": "single",
    "Married-civ-spouse": "married",
    "Married-spouse-absent": "married",
    "Married-AF-spouse": "married",
    "Divorced": "divorced",
    "Separated": "divorced",
    "Widowed": "widowed"
}
adult1["marital-status"] = adult1["marital-status"].map(dic)
adult2["marital-status"] = adult2["marital-status"].map(dic)

# les différentes valeurs du colonne adult1.marital-status après mappage
adult1["marital-status"].unique()

array(['single', 'married', 'divorced', nan], dtype=object)

On remplace les valeurs de "sex" dans la table "adult1": (Female, Male) par (F, M) respectivement.

Aussi, on remplace les valeurs de "class" dans la table "adult1": (<=50K, >50K) par (N, Y) respectivement.
    

In [722]:
adult1["sex"] = adult1["sex"].map({"Female": "F", "Male": "M"})
adult1["class"] = adult1["class"].map({"<=50K": "N", ">50K": "Y"})

In [723]:
# dimension de la table adult1
adult1.shape

(50, 7)

In [724]:
# dimension de la table adult2
adult2.shape

(50, 7)

In [725]:
# dimension de la table adult34
adult34.shape

(94, 7)

In [726]:
# fusionner les tables
adult = pandas.concat([adult1, adult2, adult34], ignore_index=True)
# dimension de la table adult
adult.shape

(194, 7)

## 3. Nétoyage des données
Avant tout, on va vérifier le nombre des valeurs indéfinies dans chaque colonne.

In [727]:
adult.isnull().sum()

age                5
workclass         10
education          1
marital-status     4
sex                2
hours-per-week     2
class              0
dtype: int64

On va, donc, nettoyer tous les enregistrements avec une valeur "NaN" sauf la colonne "age", on va la traiter autrement.

In [728]:
adult.dropna(subset=["workclass", "education", "marital-status", "sex", "hours-per-week", "class"], inplace=True)
adult.isnull().sum()

age               3
workclass         0
education         0
marital-status    0
sex               0
hours-per-week    0
class             0
dtype: int64

Pour les valeurs de absentes de "age", on va faire un lissage par moyenne:
- Transformer les valeurs de "age" comme numériques
- On regroupe les enregistrements par "class" et "education". Ceci en supposant que les individus avec la même classe et le même niveau d'éducation ont le même age. Si on veut être plus sûr, on peut tracer des graphes entre "age" et les autres attributs.
- On calcule la moyenne et l'arrondir
- On l'affecte aux valeurs indéfinies


In [729]:
adult["age"] = pandas.to_numeric(adult["age"])
adult["age"] = adult.groupby(["class", "education"])["age"].transform(lambda x: x.fillna(int(round(x.mean()))))
adult.isnull().sum()

age               0
workclass         0
education         0
marital-status    0
sex               0
hours-per-week    0
class             0
dtype: int64

## 4. Transformation des données

Pour transformer les données, on va utiliser la bibliothèque **scikit-learn**.

### 4.1. Encodage des caractéristiques nominales

Dans Scikit-learn, les caractéristiques nominales ne sont pas utilisables telles qu'elles sont. On doit les encoder vers des valeurs numériques. 

### 4.1.1. Encodage ordinal
C'est le fait d'affecter des entiers à chaque valeur 



In [730]:
adult["education"].head(6)

0    Bachelors
1    Bachelors
2      HS-grad
4    Bachelors
5      Masters
6          9th
Name: education, dtype: object

In [731]:
from sklearn import preprocessing
ord_enc = preprocessing.OrdinalEncoder()
# le résultat c'est un numpy.ndarray
education_enc = ord_enc.fit_transform(adult[["education"]])
education_enc[:6,]

array([[6.],
       [6.],
       [8.],
       [6.],
       [9.],
       [3.]])

### 4.1.2. Encodage One-Hot
Des algorithmes d'apprentissage qui nécessitent des valeurs numériques et pas des catégories (comme les réseaux de neurones) ne marchent pas bien avec l'encodeur ordinal. Un autre encodeur est OneHot qui transforme une caractéristique vers plusieurs selon le nombre de ces valeurs.

On va transformer la colonne "sex" vers deux colonnes "M" et "F"

In [732]:
adult["sex"].head(6)

0    M
1    M
2    M
4    F
5    F
6    F
Name: sex, dtype: object

In [733]:
onehot_enc = preprocessing.OneHotEncoder()
# le résultat c'est un numpy.ndarray
sex_enc = onehot_enc.fit_transform(adult[["sex"]])
sex_enc.toarray()[:6,]

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

### 4.2. Normalisation

#### 4.2.1. Mise à l'échelle min-max

La mise en échelle min-max transforme chaque valeur numérique *x* vers une autre valeur *x' ∈ [0,1]* en utilisant la valeur minimale et la valeur maximale dans les données. Cette normalisation conserve la distance proportionnelle entre les valeurs d'une caractéristique.

Supposant, on veut créer une colonne qui représente les proportions de "hours-per-week".

In [734]:
adult["hours-per-week"] = pandas.to_numeric(adult["hours-per-week"])
adult["hours-per-week"].head(3)

0    40.0
1    13.0
2    40.0
Name: hours-per-week, dtype: float64

On utilise *MinMaxScaler*. Il n'accepte pas des colonnes, donc on utilise double crochets pour avoir la colonne comme une dataframe

In [735]:
min_max_scaler = preprocessing.MinMaxScaler()
# le résultat c'est un numpy.ndarray
hours_per_week_prop = min_max_scaler.fit_transform(adult[["hours-per-week"]])
hours_per_week_prop[:3,]

array([[0.49367089],
       [0.15189873],
       [0.49367089]])

In [736]:
# pour ajouter la nouvelle caractéristique au dataframe
adult["hours-per-week-prop"] = hours_per_week_prop
adult.head(3)

,age,workclass,education,marital-status,sex,hours-per-week,class,hours-per-week-prop
0,39.0,State-gov,Bachelors,single,M,40.0,N,0.493671
1,50.0,Self-emp-not-inc,Bachelors,married,M,13.0,N,0.151899
2,38.0,Private,HS-grad,divorced,M,40.0,N,0.493671


#### 4.2.2. Z-score

Le Z-score est utilisé pour assurer que la distribution d'une caractéristique ait une moyenne = 0 et un écart type = 1.

Supposant, on veut appliquer cette transformation sur l'age pour avoir une autre caractéristique

In [737]:
adult["age"].head(3)

0    39.0
1    50.0
2    38.0
Name: age, dtype: float64

In [738]:
std_scaler = preprocessing.StandardScaler()
# le résultat c'est un numpy.ndarray
age_normal = min_max_scaler.fit_transform(adult[["age"]])
age_normal[:3,]

array([[0.3442623 ],
       [0.52459016],
       [0.32786885]])

### 4.3. Binarisation

Supposant, on veut avoir une caractéristique "jeune" (age inférieure à 40)

In [739]:
adult["age"].head(10)

0     39.0
1     50.0
2     38.0
4     28.0
5     37.0
6     49.0
7     52.0
8     31.0
9     42.0
10    43.0
Name: age, dtype: float64

In [740]:
binarizer = preprocessing.Binarizer(threshold=40)
# le résultat c'est un numpy.ndarray
age_bin = binarizer.fit_transform(adult[["age"]])
age_bin[:10,]

array([[0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.]])